### Apply the Gauss-Seidel method to solve
- 𝟓𝒙𝟏−𝟐𝒙𝟐+𝟑𝒙𝟑=−𝟏, −𝟑𝒙𝟏+𝟗𝒙𝟐+𝒙𝟑=𝟐, 𝟐𝒙𝟏−𝒙𝟐−𝟕𝒙𝟑=𝟑
- Solve once without vectorization then use vectorize implementation.
- Check for convergance.
- Use different tolerence and see the difference between the two methods. e.g. tol: 0.01,0.001,0.0001 ... etc.

In [1]:
import numpy as np
from scipy.linalg import lu

In [2]:
#sufficient but not necessary
def dominant_matrix(A):
  diagonal = []
  off_diagonal = []
  for i in range(A.shape[0]):
    diagonal.append(int(abs(A[i][i])))
    off_diagonal.append(int(sum(abs(A[i])) - abs(A[i][i])))
  counter = 0
  for i in range(len(diagonal)):
    if (diagonal[i] <=  off_diagonal[i]):
      counter +=1
  if counter == 0:
    state = "matrix is strictly diagonally dominant"
  elif counter == len(diagonal):
    state = "matrix is not  dominant"
  else:
    state = "matrix is diagonally dominant"
  return diagonal, off_diagonal, state

In [93]:
#Not vectorized
def gauss_seidel(A,y, epsilon):
  # 1. get values of x
  for i in range(A.shape[0]):
    y[i] = y[i] / A[i][i]
    A[i] = A[i] / A[i][i]
    A[i][i] = 0
  #2. assume x = [0,0,0]
  x = np.zeros(A.shape[1]).reshape(-1,1)
  #3. get values of x till convergence
  print(f"Iteration results with epsilon = {epsilon}")
  print("k \tx1 \tx2 \tx3")
  for k in range(20):
    x_prev = np.copy(x)
    for i in range(A.shape[0]):
        x[i] = (np.dot(-1* A[i] ,x)) + y[i]

    print(f"{k+1} \t{round(x[0][0],4 )} \t{round(x[1][0],4 )} \t{round(x[2][0],4 )}")
    if np.linalg.norm(x-x_prev) < epsilon:
      print("Converged!")
      break

In [94]:
A = np.array([[5, -2, 3],
     [-3, 9, 1],
     [2, -1, -7]]).astype('float64')
y = np.array([[-1], [2], [3]]).astype('float64')
diagonal, off_diagonal, state = dominant_matrix(A)
print(f"diagonal: {diagonal}")
print(f"off diagonal: {off_diagonal}")
print(state)

diagonal: [5, 9, 7]
off diagonal: [5, 4, 3]
matrix is diagonally dominant


In [95]:
gauss_seidel(A,y, 0.00001)

Iteration results with epsilon = 1e-05
k 	x1 	x2 	x3
1 	-0.2 	0.1556 	-0.5079
2 	0.167 	0.3343 	-0.4286
3 	0.1909 	0.3335 	-0.4217
4 	0.1864 	0.3312 	-0.4226
5 	0.1861 	0.3312 	-0.4227
6 	0.1861 	0.3312 	-0.4227
7 	0.1861 	0.3312 	-0.4227
Converged!


In [100]:
# vectorized
def gauss_seidel_vectorized(A,y, epsilon):
  U = np.triu(A,1)
  L = np.tril(A,-1)
  D = np.diag(np.diag(A))

  T = np.linalg.inv((D+L)) @ (-1*U)
  c = np.linalg.inv((D+L)) @ y
  x = np.zeros(A.shape[1]).reshape(-1,1)

  # spectral radius  
  print("spectral radius:", np.max(np.abs((np.linalg.eig(T)[0]))))

  print(f"Iteration results with epsilon = {epsilon}")
  print("k \tx1 \tx2 \tx3")
  for k in range(20):
    x_prev = x
    x = T @ x_prev + c

    print(f"{k+1} \t{round(x[0][0],4 )} \t{round(x[1][0],4 )} \t{round(x[2][0],4 )}")
    if np.linalg.norm(x-x_prev) < epsilon:
      print("Converged!")
      break

In [101]:
A = np.array([[5, -2, 3],
     [-3, 9, 1],
     [2, -1, -7]]).astype('float64')
y = np.array([[-1], [2], [3]]).astype('float64')
gauss_seidel_vectorized(A,y ,0.00001)

spectral radius: 0.11268723396380219
Iteration results with epsilon = 1e-05
k 	x1 	x2 	x3
1 	-0.2 	0.1556 	-0.5079
2 	0.167 	0.3343 	-0.4286
3 	0.1909 	0.3335 	-0.4217
4 	0.1864 	0.3312 	-0.4226
5 	0.1861 	0.3312 	-0.4227
6 	0.1861 	0.3312 	-0.4227
7 	0.1861 	0.3312 	-0.4227
Converged!


### Use np.linalg.solve() to solve the previous system and compare the results.

In [102]:
np.linalg.solve(A, y)

array([[ 0.18611987],
       [ 0.33123028],
       [-0.42271293]])